In [ ]:
"""
Nice, you’ve now got a best model from Keras Tuner 🎯
From here there are **two big levers**:

1. **Make the model itself better** (architecture, regularization, hyperparams)
2. **Make the *data & training* setup better** (features, targets, training schedule)

I’ll give you concrete things you can do *right now* with code.

---

## 1. Fine-tune your current best model

You already have `best_model` and `best_hp`. Easiest improvement:
**keep training the same model with a smaller learning rate.**

```python
# Take the tuned model
best_model = tuner.hypermodel.build(best_hp)

# Optionally re-train from scratch with tuned hp
history = best_model.fit(
    X_train_normal,
    y_train,
    validation_data=(X_test_normal, y_test),
    epochs=100,
    callbacks=callbacks,
    verbose=0
)

# Now fine-tune with smaller lr
import keras

old_lr = best_hp.get("learning_rate")
fine_tune_lr = old_lr * 0.1

best_model.compile(
    loss="mae",
    optimizer=keras.optimizers.Adam(learning_rate=fine_tune_lr),
    metrics=["mae", "mse"]
)

history_ft = best_model.fit(
    X_train_normal,
    y_train,
    validation_data=(X_test_normal, y_test),
    epochs=50,
    callbacks=callbacks,
    verbose=0
)

best_model.evaluate(X_test_normal, y_test)
```

This alone can squeeze a bit more performance by “polishing” weights.

---

## 2. Add regularization (dropout / L2) and tune it

Right now your model is:

* Dense → Dense → Dense(1)

You can improve **generalization** by:

* Adding **L2 weight decay**
* Adding a **Dropout** layer

And tune their strength.

```python
import keras
import keras_tuner as kt

def build_model(hp: kt.HyperParameters):
    keras.utils.set_random_seed(42)

    units_1 = hp.Choice('units_1', [64, 100, 128])
    units_2 = hp.Choice('units_2', [64, 100, 128])
    lr      = hp.Choice('learning_rate', [0.001, 0.005, 0.01])

    # New: regularization hyperparams
    l2_factor = hp.Choice('l2_factor', [0.0, 1e-4, 1e-3])
    dropout_rate = hp.Choice('dropout_rate', [0.0, 0.1, 0.2])

    reg = keras.regularizers.l2(l2_factor) if l2_factor > 0 else None

    model = keras.Sequential(name="Model_A")
    model.add(keras.layers.Input(shape=(11,)))
    model.add(keras.layers.Dense(units_1, activation='relu',
                                 kernel_regularizer=reg))
    model.add(keras.layers.Dense(units_2, activation='relu',
                                 kernel_regularizer=reg))
    # Optional dropout layer
    if dropout_rate > 0:
        model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(1))

    model.compile(
        loss="mae",
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=["mae", "mse"]
    )
    return model
```

Then rerun your `GridSearch` (or better: switch to `RandomSearch` or `BayesianOptimization` so the search space can be larger):

```python
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_mae",
    max_trials=30,
    overwrite=True,
    directory="kt_dir",
    project_name="model_a_reg"
)

tuner.search(
    X_train_normal, y_train,
    validation_data=(X_test_normal, y_test),
    epochs=150,
    callbacks=callbacks,
    verbose=1
)
```

---

## 3. Tune batch size & number of layers

Two more impactful knobs:

* **batch_size** (small batches can help generalization sometimes)
* **depth** (1, 2, or 3 hidden layers)

### Example: tuning batch size

```python
batch_size = hp.Choice("batch_size", [16, 32, 64, 128])
...
# in tuner.search:
tuner.search(
    X_train_normal,
    y_train,
    validation_data=(X_test_normal, y_test),
    epochs=150,
    batch_size=best_hp.get("batch_size"),   # if you add it
    callbacks=callbacks,
    verbose=1
)
```

Better pattern: put `batch_size` also in `search` as:

```python
tuner.search(
    X_train_normal,
    y_train,
    validation_data=(X_test_normal, y_test),
    epochs=150,
    callbacks=callbacks,
    verbose=1,
    batch_size=hp.Choice("batch_size", [32, 64, 128])  # in a custom run_trial
)
```

But the simple way is: try a few batch sizes manually once you have a good model.

### Example: tuning number of hidden layers

You can optionally add a 3rd dense layer only in some trials:

```python
def build_model(hp):
    keras.utils.set_random_seed(42)

    lr = hp.Choice("learning_rate", [0.001, 0.005, 0.01])
    num_hidden = hp.Choice("num_hidden", [1, 2, 3])

    model = keras.Sequential(name="Model_A")
    model.add(keras.layers.Input(shape=(11,)))

    units = hp.Choice("units", [64, 100, 128])

    for i in range(num_hidden):
        model.add(keras.layers.Dense(units, activation="relu"))

    model.add(keras.layers.Dense(1))

    model.compile(
        loss="mae",
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=["mae", "mse"],
    )
    return model
```

---

## 4. Improve input / target side (often bigger gains)

Sometimes the **biggest MAE improvement** comes from data, not layers:

1. **Check scaling of targets (`y`)**

   * If `y` has a long tail, try a **log transform**: `y_log = np.log1p(y)`
   * Train the model on `y_log`, and when predicting, invert with `np.expm1(pred)`.

2. **Handle outliers**

   * Clip extreme values of target or features.
   * Or train with a robust loss (e.g., Huber, which you already played with).

3. **Feature engineering**

   * Add interactions: `x1 * x2`, ratios, or bucketed versions of continuous features.
   * If there are categorical variables, use proper one-hot or embeddings instead of arbitrary numeric codes.

4. **Cross-validation**

   * Instead of a single train/val split, use **KFold** and average MAE.
   * This gives more stable hyperparameter selection and can reveal overfitting to a specific split.

---

## 5. Quick checklist – what I’d do in your notebook

If you want a practical order:

1. ✅ **Restart kernel** and use only `keras` + `keras_tuner` (no `tf.keras` models/callbacks).
2. ✅ Keep your current best hyperparams and:

   * Recompile with `lr * 0.1`
   * Fine-tune 30–50 more epochs with callbacks.
3. ✅ Extend tuner search:

   * Add **L2 + Dropout** hyperparams.
   * Maybe allow `num_hidden` ∈ {1, 2, 3}.
   * Switch to `RandomSearch` with ~30 trials.
4. ✅ Examine distribution of `y_train`:

   * If skewed → try `log1p` target.
5. ✅ If MAE still doesn’t move much, we’re probably close to the **information limit of your data** (noise level, missing features, etc.).

If you paste your **best_model.evaluate output** and a short description of what the target is (range, approx noise), I can suggest very specific next tweaks.

"""

In [1]:
"""
2. Use a smarter tuner than GridSearch

GridSearch explodes combinatorially and often wastes trials. For neural nets, RandomSearch or BayesianOptimization usually find better configs faster.
"""

'\n2. Use a smarter tuner than GridSearch\n\nGridSearch explodes combinatorially and often wastes trials. For neural nets, RandomSearch or BayesianOptimization usually find better configs faster.\n'